# Q3: Learned Nodes Decision Boundaries

In [ ]:
import random
import warnings

warnings.filterwarnings("ignore")
random.seed(1234)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from models.mlp_model import MLP
from sklearn.metrics import accuracy_score
from training_testing.mse_testing import plot_decision_boundary
from training_testing.mse_training import train
from utilities import load_data, sigmoid, tanh

## Loading Center Surround Data

In [ ]:
X_train, y_train = load_data("data/center_surround_train.csv")
X_valid, y_valid = load_data("data/center_surround_valid.csv")
X_test, y_test = load_data("data/center_surround_test.csv")
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

csv_filename = "results/center_surround/mse_hyperparameter_results.csv"
dataset = "center_surround"

## Training Model

In [ ]:
lr = 0.01
batch_size = 16
k = 15
epochs = 150

In [ ]:
model = MLP(input_size=X_train.shape[1], hidden_size=k)

In [ ]:
_, _, _, _ = train(
    model,
    X_train,
    y_train,
    X_valid,
    y_valid,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)

## Overall Decision Boundary

In [ ]:
test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test.squeeze(), test_pred)

print(f"Test accuracy for k={k}, Batch={batch_size}, LR={lr}: {test_accuracy}")

In [ ]:
plot_decision_boundary(model, X_test, y_test, test_pred, dataset)

## Layer-wise Decision Boundary

In [ ]:
def predict_hidden_layers(X, W1, W2):
    X_bias = np.insert(X, 0, 1, axis=1)
    Z1 = np.dot(X_bias, W1)
    A1 = tanh(Z1)
    A1_bias = np.insert(A1, 0, 1, axis=1)
    Z2 = A1_bias * W2.reshape(1, -1)
    output = sigmoid(Z2)

    predictions = (output > 0.5).astype(int)
    return predictions

In [ ]:
def plot_decision_boundaries_for_hidden_layers(model, X, y, W1, W2, dataset):
    hidden_layer_predictions = predict_hidden_layers(X, W1, W2)
    num_nodes = hidden_layer_predictions.shape[1]

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    df_test = pd.DataFrame(X, columns=["Feature 1", "Feature 2"])
    df_test["Label"] = y.squeeze()
    df_test["Label"] = df_test["Label"].astype(int)

    for i in range(num_nodes):
        plt.figure(figsize=(10, 6))

        Z = predict_hidden_layers(np.c_[xx.ravel(), yy.ravel()], W1, W2)[:, i]
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.8, cmap="RdBu_r")
        sns.scatterplot(
            data=df_test,
            x="Feature 1",
            y="Feature 2",
            hue="Label",
            palette=["blue", "red"],
        )

        # Highlight incorrect predictions for the i-th node
        incorrect_predictions = hidden_layer_predictions[:, i] != y.squeeze()
        incorrect_points = df_test[incorrect_predictions]
        plt.scatter(
            incorrect_points["Feature 1"],
            incorrect_points["Feature 2"],
            facecolors="none",
            edgecolors="black",
            s=100,
            label="Incorrectly Classified",
        )

        plt.title(f"Node {i+1} Decision Boundary")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")
        plt.legend(title="Legend", loc="upper right")
        plt.savefig(f"results/{dataset}/node_{i+1}_decision_boundary.png")
        plt.show()

In [ ]:
plot_decision_boundaries_for_hidden_layers(
    model, X_test, y_test, model.W1, model.W2, dataset
)